### Import

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import psycopg2
import pandas.io.sql as psql

ImportError: No module named psycopg2

### connect_to_dw

In [3]:
conn = ""
try:
        conn = psycopg2.connect(dbname='dw_blocketdb_ch', user='bnbiuser', host='200.29.173.148', password='VE1bi@BN112AzLkOP')
except:
        logger.info('ERROR: I am unable to connect to the database')
        print
        "I am unable to connect to the database"

### SQL Query

In [4]:
query = """
select *,
case when im.ad_id_nk is null then 'no' else 'yes' end as has_a_photo
from ods.ad a inner join
ods.category c on c.category_id_pk = a.category_id_fk
left join
stg.temp_images_per_ad im on im.ad_id_nk = a.ad_id_nk
where a.approval_date::date between '2018-08-01' and '2018-10-31'
"""
df_sql = psql.read_sql(query, conn)
#conn.close()
#return df_sql

In [5]:
print df_sql

         ad_id_pk  ad_type_id_fk  category_id_fk  platform_id_fk  \
0        69389975              1               7               7   
1        69386281              1              47               1   
2        69367451              1               7               2   
3        69374786              1               7               7   
4        69380689              1               7               1   
5        69384623              1               9               2   
6        69382995              1               9               2   
7        69386796              1              30               7   
8        69384001              1              30               2   
9        69367832              1              29               2   
10       69380812              1              30               2   
11       69381478              1              28               2   
12       69396537              1              28               2   
13       69369873              1              29

### Words distribution

In [6]:
count = df_sql['body'].str.split().apply(len).value_counts()
count.index = count.index.astype(str) + ' words:'
count.sort_index(inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(count)

0 words:          5
1 words:      13074
10 words:     85020
100 words:     2438
101 words:     2077
102 words:     1847
103 words:     2189
104 words:     1851
105 words:     1804
106 words:     1929
107 words:     1643
108 words:     1970
109 words:     1427
11 words:     83539
110 words:     1531
111 words:     1853
112 words:     1467
113 words:     2896
114 words:     1438
115 words:     1309
116 words:     1630
117 words:     1376
118 words:     1935
119 words:     1459
12 words:     80292
120 words:     1292
121 words:     1742
122 words:     1374
123 words:     1302
124 words:     1521
125 words:     2518
126 words:     4069
127 words:     1262
128 words:     1099
129 words:     1174
13 words:     79011
130 words:     1079
131 words:     1053
132 words:     1051
133 words:     1076
134 words:     1122
135 words:      975
136 words:     1080
137 words:      890
138 words:      884
139 words:      918
14 words:     74398
140 words:      991
141 words:      850
142 words:      894


### C Description per categories

In [351]:
count_body = df_sql['body'].str.split().apply(len)
count_subect = df_sql['subject'].str.split().apply(len)
df_category = df_sql['category_id_fk']

df_merge_bs = pd.concat([count_body, count_subect], axis=1, sort=False)

df_merge_bs = pd.concat([df_merge_bs, df_category], axis=1, sort=False)
df_merge_bs['total_words'] = df_merge_bs.apply(lambda row: row.body + row.subject, axis=1)
df_merge_bs = df_merge_bs[['category_id_fk','total_words']]

df_merge_bs
#print df_body

#print df_subject

,category_id_fk,total_words
0,7,49
1,34,39
2,29,36
3,12,23
4,30,19
5,7,25
6,28,30
7,35,13
8,28,31
9,37,25


In [352]:
cat_id=[1,2,3,4,5,6,7,8,9,10,11,12,19,20,21,22,23,24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
df_words_cat=[]
for x in cat_id:
    df_words_cat.append(df_merge_bs[df_merge_bs['category_id_fk']==x].quantile([.25,.75]))
    

In [353]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_words_cat)


[      category_id_fk  total_words
0.25             NaN          NaN
0.75             NaN          NaN,       category_id_fk  total_words
0.25             NaN          NaN
0.75             NaN          NaN,       category_id_fk  total_words
0.25             NaN          NaN
0.75             NaN          NaN,       category_id_fk  total_words
0.25             NaN          NaN
0.75             NaN          NaN,       category_id_fk  total_words
0.25             NaN          NaN
0.75             NaN          NaN,       category_id_fk  total_words
0.25             NaN          NaN
0.75             NaN          NaN,       category_id_fk  total_words
0.25             7.0         22.0
0.75             7.0         63.0,       category_id_fk  total_words
0.25             8.0         17.0
0.75             8.0         40.0,       category_id_fk  total_words
0.25             9.0         16.0
0.75             9.0         38.0,       category_id_fk  total_words
0.25            10.0         16.0
0.75

In [179]:
count_body.quantile([.25,.75])

0.25    15.5
0.75    26.0
Name: body, dtype: float64

### C Price per categories

In [354]:
df_query = pd.DataFrame(df_sql)
df_price = pd.DataFrame()

In [355]:
df_price=df_query[['category_id_fk','price']]
df_price


,category_id_fk,price
0,7,4200000.0
1,34,NaN
2,29,370000.0
3,12,8000000.0
4,30,130000.0
5,7,6300000.0
6,28,20000.0
7,35,800000.0
8,28,20000.0
9,37,15000.0


In [260]:
df_price[df_price['category_id_fk']==31].quantile([.1,.9])

,category_id_fk,price
0.1,31.0,10000.0
0.9,31.0,225000.0


In [356]:
cate_name=[1,2,3,4,5,6,7,8,9,10,11,12,19,20,21,22,23,24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
df_price_cat=[]
for x in cate_name:
    df_price_cat.append(df_price[df_price['category_id_fk']==x].quantile([.1,.9]))

In [1]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_price_cat)

NameError: name 'pd' is not defined

### Photos

In [371]:
df_query = pd.DataFrame(df_sql)
df_photo = pd.DataFrame()

In [373]:
#df_photo=df_query[['category_id_fk','has_a_photo','photos','mean_resol']]
df_photo=df_query[['category_id_fk','photos']]
df_photo

,category_id_fk,photos
0,34,NaN
1,12,4.0
2,7,6.0
3,7,12.0
4,20,1.0
5,29,3.0
6,40,4.0
7,19,6.0
8,37,NaN
9,7,4.0


In [374]:
cate_name=[1,2,3,4,5,6,7,8,9,10,11,12,19,20,21,22,23,24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
df_photo_cat=[]
for x in cate_name:
    df_photo_cat.append(df_photo[df_photo['category_id_fk']==x].quantile([.5]))

In [375]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_photo_cat)

[     category_id_fk  photos
0.5             NaN     NaN,      category_id_fk  photos
0.5             NaN     NaN,      category_id_fk  photos
0.5             NaN     NaN,      category_id_fk  photos
0.5             NaN     NaN,      category_id_fk  photos
0.5             NaN     NaN,      category_id_fk  photos
0.5             NaN     NaN,      category_id_fk  photos
0.5             7.0     6.0,      category_id_fk  photos
0.5             8.0     5.0,      category_id_fk  photos
0.5             9.0     4.0,      category_id_fk  photos
0.5            10.0     4.0,      category_id_fk  photos
0.5            11.0     2.0,      category_id_fk  photos
0.5            12.0     4.0,      category_id_fk  photos
0.5            19.0     3.0,      category_id_fk  photos
0.5            20.0     3.0,      category_id_fk  photos
0.5            21.0     3.0,      category_id_fk  photos
0.5            22.0     2.0,      category_id_fk  photos
0.5            23.0     2.0,      category_id_fk  photos
0.

In [402]:
df_resol=df_query[['category_id_fk','mean_resol']]

df_resol = df_resol[df_resol['mean_resol']>0]

cate_name=[1,2,3,4,5,6,7,8,9,10,11,12,19,20,21,22,23,24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]

df_resol.groupby(df_resol.category_id_fk)[['mean_resol']].median()

,mean_resol
category_id_fk,
7,272.0
8,240.0
9,230.0
10,205.0
11,209.0
12,230.0
19,172.0
20,230.0
21,172.0
